In [1]:
# !pip install open3d
import open3d as o3d
import numpy as np
import sys

sys.path.append('./data/')
from camera import *
from dataloader import *

sys.path.append('./algorithm/')
from measurement import *
from pose import *
from update import *
from prediction import *

from pipeline import *

import matplotlib.pyplot as plt
import time

In [2]:
trajectory = np.load('trajectory.npy')
print(trajectory.shape)

(798, 8)


In [3]:
print(trajectory)

[[ 1.30503110e+09  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 1.30503110e+09  3.54141684e-03 -1.79871187e-02 ...  1.27855963e-03
   1.40839448e-03  9.99989467e-01]
 [ 1.30503110e+09 -5.68590333e-03  4.43203487e-03 ...  3.18983532e-03
   3.70107560e-04  9.99958804e-01]
 ...
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]]


In [5]:
with open('trajectory.txt', 'w') as f:
    f.write('# Output trajectory with each line: timestamp tx ty tz qx qy qz qw\n')
    for i in range(trajectory.shape[0]):
        for j in range(trajectory.shape[1]):
            f.write('{:.4f} '.format(trajectory[i][j]))
        f.write('\n')

In [2]:
vbg = o3d.t.geometry.VoxelBlockGrid.load('./results/TSDF.npz')

In [3]:
visualize_vbg_o3d(vbg, 180, 0, 0)

PointCloud on CPU:0 [458158 points (Float32)].
Attributes: normals (dtype = Float32, shape = {458158, 3}).
FEngine (64 bits) created at 0x7feb01000000 (threading is enabled)
FEngine resolved backend: OpenGL


[error] GLFW error: Cocoa: Failed to find service port for display


In [5]:
locations = trajectory[:,:3]

In [6]:
locations *= 1000
print(locations.dtype)
print(locations.shape)

float64
(798, 3)


In [7]:
visualize_point_cloud_numpy(locations, 0, 0, 0)

PointCloud with 798 points.
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [4]:
kf.first_frame()
timer = PerfTimer()

while kf.has_next_frame():
    timer.startMeasure("Overall frame")
    kf.next_frame()
    timer.stopMeasure()

KinectFusion pipeline total frame number: 798
Running frame: 0


KeyboardInterrupt: 